# Query openMDM Server 

This Notebook shows how to query instance data of your openMDM Server using the HTTP API.


Configure the base_url to your openMDM installation. 
You can then open the **Web client** to explore your data interactively or open the **swagger OpenAPI contract** to inspect the API. 

In [1]:
#base_url ='http://docker.peaksolution.local:82/org.eclipse.mdm.nucleus'
base_url ='https://docker.peak-solution.de:10031/org.eclipse.mdm.nucleus'


print(f"Web Client: {base_url}")
print(f"OpenAPI: {base_url}/swagger.html")

Web Client: https://docker.peak-solution.de:10031/org.eclipse.mdm.nucleus
OpenAPI: https://docker.peak-solution.de:10031/org.eclipse.mdm.nucleus/swagger.html


## Dependencies for this notebook

In [2]:
import requests
import sys
import json
import mdm_pb2 as mdm
from google.protobuf.timestamp_pb2 import Timestamp

## Establish session

In [3]:

# login at glassfish http interface with form based authentication
session = requests.Session()
session.headers={'Content-Type': 'application/json', 'Accept': 'application/json'}

r = session.post(base_url + '/j_security_check', data={'j_username': 'Demo', 'j_password': 'mdm'}, headers={'Content-Type': 'application/x-www-form-urlencoded'})

r.raise_for_status() # throw if failed

The openMDM API is a session based API. The session object contains the session cookie. Close this session to release the connection license. Otherwise the session will be auto closed after 30 minutes of inactivity.

In [4]:
r = session.get(f'{base_url}/mdm/datasources')
r.raise_for_status() # throw if failed
r.json()

['NVHDEMO', 'CRASHDEMO', 'BLANKDEMO', 'ADASDEMO', 'FDXDEMO']

Now we could see the available data sources. Now we pick one to go on.

In [5]:
SOURCENAME=r.json()[0]
SOURCENAME

'NVHDEMO'

## Retrieve Project instances

### 📁Retrieve Project instances by query

Retrieve Projects from datasource. There are different ways.
We start with the generic one `query`.

In [6]:
query = {
    "filters": [
        {
            "sourceName": SOURCENAME,
            "filter": "",
            "searchString": ""
        }
    ],
    "columns": [
        "Project.Name"
    ],
    "resultType": "Project"
}
r = session.post(f'{base_url}/mdm/query',json=query)
r.raise_for_status()
projects = r.json()

project_names = [row['columns'][0]['value'] for row in projects['rows']]
print(f'Project Names: {project_names}')

projects

Project Names: ['PMV 2PV', 'PMV Model P', 'PMV Summit']


{'rows': [{'source': 'NVHDEMO',
   'type': 'Project',
   'id': '1',
   'columns': [{'type': 'Project', 'attribute': 'Name', 'value': 'PMV 2PV'}]},
  {'source': 'NVHDEMO',
   'type': 'Project',
   'id': '2',
   'columns': [{'type': 'Project',
     'attribute': 'Name',
     'value': 'PMV Model P'}]},
  {'source': 'NVHDEMO',
   'type': 'Project',
   'id': '3',
   'columns': [{'type': 'Project',
     'attribute': 'Name',
     'value': 'PMV Summit'}]}],
 'totalRecords': 3}

Now the filter can be extended or additional columns can be added.
Here we add the condition that `Project.Name` should be like `PM*` and an additional column `Project.Id` was picked.

In [7]:
query = {
    "filters": [
        {
            "sourceName": SOURCENAME,
            "filter": "Project.Name lk 'PM*'",
            "searchString": ""
        }
    ],
    "columns": [
        "Project.Name",
        "Project.Id",
    ],
    "resultType": "Project"
}
r = session.post(f'{base_url}/mdm/query',json=query)
r.raise_for_status()
projects = r.json()

project_names = []
for row in projects['rows']:
    for column in row['columns']:
        if 'Name' == column['attribute']:
            project_names.append(column['value'])
print(f'Project Names: {project_names}')

projects

Project Names: ['PMV 2PV', 'PMV Model P', 'PMV Summit']


{'rows': [{'source': 'NVHDEMO',
   'type': 'Project',
   'id': '1',
   'columns': [{'type': 'Project', 'attribute': 'Name', 'value': 'PMV 2PV'},
    {'type': 'Project', 'attribute': 'Id', 'value': '1'}]},
  {'source': 'NVHDEMO',
   'type': 'Project',
   'id': '2',
   'columns': [{'type': 'Project',
     'attribute': 'Name',
     'value': 'PMV Model P'},
    {'type': 'Project', 'attribute': 'Id', 'value': '2'}]},
  {'source': 'NVHDEMO',
   'type': 'Project',
   'id': '3',
   'columns': [{'type': 'Project', 'attribute': 'Name', 'value': 'PMV Summit'},
    {'type': 'Project', 'attribute': 'Id', 'value': '3'}]}],
 'totalRecords': 3}

The REST API also contains an entry point for all objects.

### 📁Project instances by entry point

The REST API also contains an entry point for all objects of a given type.

In [8]:
r = session.get(f'{base_url}/mdm/environments/{SOURCENAME}/projects')
r.raise_for_status()
projects = r.json()

project_names = []
for row in projects['data']:
    for column in row['attributes']:
        if 'Name' == column['name']:
            project_names.append(column['value'])
print(f'Project Names: {project_names}')

projects

Project Names: ['PMV 2PV', 'PMV Model P', 'PMV Summit']


{'type': 'Project',
 'data': [{'name': 'PMV 2PV',
   'id': '1',
   'type': 'Project',
   'sourceType': 'Project',
   'sourceName': 'NVHDEMO',
   'attributes': [{'name': 'MimeType',
     'value': 'application/x-asam.aotest',
     'unit': '',
     'dataType': 'STRING'},
    {'name': 'Name', 'value': 'PMV 2PV', 'unit': '', 'dataType': 'STRING'}],
   'relations': []},
  {'name': 'PMV Model P',
   'id': '2',
   'type': 'Project',
   'sourceType': 'Project',
   'sourceName': 'NVHDEMO',
   'attributes': [{'name': 'MimeType',
     'value': 'application/x-asam.aotest',
     'unit': '',
     'dataType': 'STRING'},
    {'name': 'Name',
     'value': 'PMV Model P',
     'unit': '',
     'dataType': 'STRING'}],
   'relations': []},
  {'name': 'PMV Summit',
   'id': '3',
   'type': 'Project',
   'sourceType': 'Project',
   'sourceName': 'NVHDEMO',
   'attributes': [{'name': 'MimeType',
     'value': 'application/x-asam.aotest',
     'unit': '',
     'dataType': 'STRING'},
    {'name': 'Name', 'value

You can also retrieve info from a single element. The returned structure is equal but only one element will be returned.

In [9]:
project_id = projects['data'][0]['id']

r = session.get(f'{base_url}/mdm/environments/{SOURCENAME}/projects/{project_id}')
r.raise_for_status()
project = r.json()

project

{'type': 'Project',
 'data': [{'name': 'PMV 2PV',
   'id': '1',
   'type': 'Project',
   'sourceType': 'Project',
   'sourceName': 'NVHDEMO',
   'attributes': [{'name': 'MimeType',
     'value': 'application/x-asam.aotest',
     'unit': '',
     'dataType': 'STRING'},
    {'name': 'Name', 'value': 'PMV 2PV', 'unit': '', 'dataType': 'STRING'}],
   'relations': []}]}

## Get Tree children

- 📁Project
  - 📁Pool
    - 📁Test
      - 📁TestStep
        - 📖Ordered Context
          - 🚗UnitUnderTest
          - 🏭TestEquipment
          - 🔀TestSequence
        - 📊Measurement
          - 📖Measured Context
            - 🚗UnitUnderTest
            - 🏭TestEquipment
            - 🔀TestSequence

### 📁Pool

Traversing down the hierarchy, the next level is called 'Pool'. To get all 'Pool'-children of our 'Project' you use the same filter syntax as before, now querying for '/pools' which belong to the same parent 'Project' by using the `project_id` - as defined by the `pool_filter`.

In [10]:
pool_filter = f'Project.Id eq {project_id}'

r = session.get(f'{base_url}/mdm/environments/{SOURCENAME}/pools', params={'filter': pool_filter})
r.raise_for_status()
pools = r.json()

pool_id = pools['data'][0]['id']

pools

{'type': 'Pool',
 'data': [{'name': 'Engine Noise Measurements',
   'id': '7',
   'type': 'Pool',
   'sourceType': 'StructureLevel',
   'sourceName': 'NVHDEMO',
   'attributes': [{'name': 'MimeType',
     'value': 'application/x-asam.aosubtest.structurelevel',
     'unit': '',
     'dataType': 'STRING'},
    {'name': 'Name',
     'value': 'Engine Noise Measurements',
     'unit': '',
     'dataType': 'STRING'}],
   'relations': []}]}

### 📁Test

For finding the 'Test'-children of the 'Pool' from above, you can follow the same pattern and attach a `test_filter` with the `pool_id`.

In [11]:
test_filter = f'Pool.Id eq {pool_id}'

r = session.get(f'{base_url}/mdm/environments/{SOURCENAME}/tests', params={'filter': test_filter})
r.raise_for_status()
tests = r.json()

test_id = tests['data'][0]['id']

tests

{'type': 'Test',
 'data': [{'name': 'EngineNoise 2PV 20191206',
   'id': '5',
   'type': 'Test',
   'sourceType': 'Test',
   'sourceName': 'NVHDEMO',
   'attributes': [{'name': 'DateClosed',
     'value': '2014-09-23T13:18:26Z',
     'unit': '',
     'dataType': 'DATE'},
    {'name': 'Description', 'value': '', 'unit': '', 'dataType': 'STRING'},
    {'name': 'MDMLinks',
     'value': '',
     'unit': '',
     'dataType': 'FILE_LINK_SEQUENCE'},
    {'name': 'DateCreated',
     'value': '2019-12-06T15:59:35Z',
     'unit': '',
     'dataType': 'DATE'},
    {'name': 'MimeType',
     'value': 'application/x-asam.aosubtest.test',
     'unit': '',
     'dataType': 'STRING'},
    {'name': 'Name',
     'value': 'EngineNoise 2PV 20191206',
     'unit': '',
     'dataType': 'STRING'}],
   'relations': [{'name': None,
     'type': 'MUTABLE',
     'entityType': 'TemplateTest',
     'contextType': None,
     'ids': ['4']},
    {'name': None,
     'type': 'MUTABLE',
     'entityType': 'Classificatio

### 📁TestStep

And for the 'TestStep'-children of the 'Test', you can apply again the exact same pattern... 

In [12]:
test_step_filter = f'Test.Id eq {test_id}'

r = session.get(f'{base_url}/mdm/environments/{SOURCENAME}/teststeps', params={'filter': test_step_filter})
r.raise_for_status()
test_steps = r.json()

test_step_id = test_steps['data'][0]['id']

test_steps

{'type': 'TestStep',
 'data': [{'name': 'EngineNoise - 100% load',
   'id': '12',
   'type': 'TestStep',
   'sourceType': 'TestStep',
   'sourceName': 'NVHDEMO',
   'attributes': [{'name': 'Sortindex',
     'value': '10',
     'unit': '',
     'dataType': 'INTEGER'},
    {'name': 'Description', 'value': '', 'unit': '', 'dataType': 'STRING'},
    {'name': 'MDMLinks',
     'value': '',
     'unit': '',
     'dataType': 'FILE_LINK_SEQUENCE'},
    {'name': 'Optional', 'value': 'true', 'unit': '', 'dataType': 'BOOLEAN'},
    {'name': 'DateCreated',
     'value': '2019-12-06T15:59:35Z',
     'unit': '',
     'dataType': 'DATE'},
    {'name': 'MimeType',
     'value': 'application/x-asam.aosubtest.teststep',
     'unit': '',
     'dataType': 'STRING'},
    {'name': 'Name',
     'value': 'EngineNoise - 100% load',
     'unit': '',
     'dataType': 'STRING'}],
   'relations': [{'name': None,
     'type': 'MUTABLE',
     'entityType': 'TemplateTestStep',
     'contextType': None,
     'ids': ['2

### 📖Context

The 'Context' contains additional meta data for a certain 'Test' respective 'TestStep'. The context is being composed of UnitUnderTest, TestEquipment and TestSequence.
The 'Context' can vary between the moment of the test being 'ordered' and the data being 'measured' (think of a sensor which needed to be exchanged), this call will return all three for measured and ordered:

- measured
    - 🚗UNITUNDERTEST
    - 🏭TESTSEQUENCE
    - 🔀TESTEQUIPMENT
- ordered
    - 🚗UNITUNDERTEST
    - 🏭TESTSEQUENCE
    - 🔀TESTEQUIPMENT

But we only print the 'measured' context...

In [13]:
r = session.get(f'{base_url}/mdm/environments/{SOURCENAME}/teststeps/{test_step_id}/contexts')
r.raise_for_status()
contexts = r.json()

context_measured = next(context["measured"] for context in contexts['data'] if "measured" in context)
context_ordered = next(context["ordered"] for context in contexts['data'] if "ordered" in context)

print(f"Context: {context_measured.keys()}")

context_measured

Context: dict_keys(['UNITUNDERTEST', 'TESTEQUIPMENT', 'TESTSEQUENCE'])


{'UNITUNDERTEST': [{'name': 'engine',
   'id': '39',
   'type': 'ContextComponent',
   'sourceType': 'engine',
   'sourceName': 'NVHDEMO',
   'attributes': [{'name': 'MimeType',
     'value': 'application/x-asam.aounitundertestpart.engine.engine',
     'unit': '',
     'dataType': 'STRING',
     'sortIndex': None,
     'readOnly': None,
     'optional': None,
     'description': None},
    {'name': 'Name',
     'value': 'engine',
     'unit': '',
     'dataType': 'STRING',
     'sortIndex': None,
     'readOnly': None,
     'optional': None,
     'description': None},
    {'name': 'manufacturer',
     'value': 'Peak Motors Ltd.',
     'unit': '',
     'dataType': 'STRING',
     'sortIndex': 20,
     'readOnly': False,
     'optional': True,
     'description': ''},
    {'name': 'model',
     'value': 'Peak 123',
     'unit': '',
     'dataType': 'STRING',
     'sortIndex': 30,
     'readOnly': False,
     'optional': True,
     'description': ''},
    {'name': 'type',
     'value': 'Co

It is also possible to retrieve a single context type, for instance UnitUnderTest.

- measured
    - 🚗UNITUNDERTEST
- ordered
    - 🚗UNITUNDERTEST


In [14]:
r = session.get(f'{base_url}/mdm/environments/{SOURCENAME}/teststeps/{test_step_id}/contexts/unitundertest')
r.raise_for_status()
contexts_unitundertest = r.json()

contexts_unitundertest

{'data': [{'measured': {'UNITUNDERTEST': [{'name': 'engine',
      'id': '39',
      'type': 'ContextComponent',
      'sourceType': 'engine',
      'sourceName': 'NVHDEMO',
      'attributes': [{'name': 'MimeType',
        'value': 'application/x-asam.aounitundertestpart.engine.engine',
        'unit': '',
        'dataType': 'STRING',
        'sortIndex': None,
        'readOnly': None,
        'optional': None,
        'description': None},
       {'name': 'Name',
        'value': 'engine',
        'unit': '',
        'dataType': 'STRING',
        'sortIndex': None,
        'readOnly': None,
        'optional': None,
        'description': None},
       {'name': 'manufacturer',
        'value': 'Peak Motors Ltd.',
        'unit': '',
        'dataType': 'STRING',
        'sortIndex': 20,
        'readOnly': False,
        'optional': True,
        'description': ''},
       {'name': 'model',
        'value': 'Peak 123',
        'unit': '',
        'dataType': 'STRING',
        'sort

### 📊Measurement

To retrieve the 'Measurement'-children of the 'TestStep', remember the query pattern used before... 

In [15]:
measurement_filter = f'TestStep.Id eq {test_step_id}'

r = session.get(f'{base_url}/mdm/environments/{SOURCENAME}/measurements', params={'filter': measurement_filter})
r.raise_for_status()
measurements = r.json()

measurement_id = measurements['data'][0]['id']

measurements

{'type': 'Measurement',
 'data': [{'name': '1/3 Octave - mics',
   'id': '156',
   'type': 'Measurement',
   'sourceType': 'MeaResult',
   'sourceName': 'NVHDEMO',
   'attributes': [{'name': 'StorageType',
     'value': 'external_only',
     'unit': '',
     'dataType': 'ENUMERATION'},
    {'name': 'Description', 'value': '', 'unit': '', 'dataType': 'STRING'},
    {'name': 'MDMLinks',
     'value': [{'identifier': 'EXTREF:$(DISC1)2023/PeakSolution-BrochurePTDM.pdf',
       'mimeType': 'application/pdf',
       'description': 'PeakSolution-BrochurePTDM.pdf',
       'fileName': 'PeakSolution-BrochurePTDM.pdf'}],
     'unit': '',
     'dataType': 'FILE_LINK_SEQUENCE'},
    {'name': 'Size', 'value': '2188', 'unit': '', 'dataType': 'LONG'},
    {'name': 'MeasurementEnd',
     'value': '2014-09-23T13:18:26Z',
     'unit': '',
     'dataType': 'DATE'},
    {'name': 'DateCreated',
     'value': '2014-09-23T13:18:26Z',
     'unit': '',
     'dataType': 'DATE'},
    {'name': 'analytic_path',
   

### 📊ChannelGroup

To retrieve the 'ChannelGroup'-children of the 'Measurement', remember the query pattern used before...

> Note: A 'ChannelGroups' is sometimes referred to as 'SubMatrix'.  

In [16]:
channelgroup_filter = f'Measurement.Id eq {measurement_id}'

r = session.get(f'{base_url}/mdm/environments/{SOURCENAME}/channelgroups', params={'filter': channelgroup_filter})
r.raise_for_status()
channelgroups = r.json()

channelgroup_id = channelgroups['data'][0]['id']

channelgroups

{'type': 'ChannelGroup',
 'data': [{'name': 'Sx:1/3 Octave(mics)',
   'id': '202',
   'type': 'ChannelGroup',
   'sourceType': 'SubMatrix',
   'sourceName': 'NVHDEMO',
   'attributes': [{'name': 'MimeType',
     'value': 'application/x-asam.aosubmatrix',
     'unit': '',
     'dataType': 'STRING'},
    {'name': 'SubMatrixNoRows',
     'value': '11',
     'unit': '',
     'dataType': 'INTEGER'},
    {'name': 'Name',
     'value': 'Sx:1/3 Octave(mics)',
     'unit': '',
     'dataType': 'STRING'}],
   'relations': []},
  {'name': 'Sy:1/3 Octave(mics)',
   'id': '201',
   'type': 'ChannelGroup',
   'sourceType': 'SubMatrix',
   'sourceName': 'NVHDEMO',
   'attributes': [{'name': 'MimeType',
     'value': 'application/x-asam.aosubmatrix',
     'unit': '',
     'dataType': 'STRING'},
    {'name': 'SubMatrixNoRows',
     'value': '165',
     'unit': '',
     'dataType': 'INTEGER'},
    {'name': 'Name',
     'value': 'Sy:1/3 Octave(mics)',
     'unit': '',
     'dataType': 'STRING'}],
   'rel

### 📈Channel

Finally we end at the 'Channel'-level.

In [17]:
channels_filter = f'ChannelGroup.Id eq {channelgroup_id}'

r = session.get(f'{base_url}/mdm/environments/{SOURCENAME}/channels', params={'filter': channels_filter})
r.raise_for_status()
channels = r.json()

channel_id = channels['data'][0]['id']

channels

{'type': 'Channel',
 'data': [{'name': 'Octave Frequency',
   'id': '946',
   'type': 'Channel',
   'sourceType': 'MeaQuantity',
   'sourceName': 'NVHDEMO',
   'attributes': [{'name': 'flags_name',
     'value': '',
     'unit': '',
     'dataType': 'STRING'},
    {'name': 'Minimum', 'value': '', 'unit': '', 'dataType': 'DOUBLE'},
    {'name': 'raw_name', 'value': '', 'unit': '', 'dataType': 'STRING'},
    {'name': 'Description', 'value': '', 'unit': '', 'dataType': 'STRING'},
    {'name': 'phys_imag_name', 'value': '', 'unit': '', 'dataType': 'STRING'},
    {'name': 'non_reference_channel_name',
     'value': '',
     'unit': '',
     'dataType': 'STRING'},
    {'name': 'Average', 'value': '', 'unit': '', 'dataType': 'DOUBLE'},
    {'name': 'Rank', 'value': '', 'unit': '', 'dataType': 'INTEGER'},
    {'name': 'Dimension',
     'value': '',
     'unit': '',
     'dataType': 'INTEGER_SEQUENCE'},
    {'name': 'MimeType',
     'value': 'application/x-asam.aomeasurementquantity',
     'uni

## Close the session

It is important to close the session to make sure the license bound to the session is freed.

In [18]:
r = session.get(f'{base_url}/mdm/logout')
r.raise_for_status()
session.close()

## License

Copyright © 2025 [Peak Solution GmbH](https://peak-solution.de)

The training material in this repository is licensed under a Creative Commons BY-NC-SA 4.0 license. See [LICENSE](../LICENSE) file for more information.